In [2]:
"""
papr-review-2023-02/notebook/oracle-manipulation.ipynb

Calculates the cost of attack to liquidate the highest LTV vault on PAPR
by manipulating the Uni V3 pool. Output is amount of PAPR needed to sell
into the Uni V3 PAPR/WETH spot pool referenced by the PAPR controller.

Ignores the PAPR controller cap on target-to-mark ratio.
"""

import typing as tp
import pandas as pd
import numpy as np
from ape import chain, Contract, networks

In [3]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [4]:
papr = Contract("0x320aAAB3038bc08317f5a4be19EA1D9608551d79")
papr

<PaprToken 0x320aAAB3038bc08317f5a4be19EA1D9608551d79>

In [5]:
pool = Contract("0x238cdFfc9097591D12F5C00136514FBE563f87BF")  # paprMEME/WETH 1% Uni V3 pool
pool

<UniswapV3Pool 0x238cdFfc9097591D12F5C00136514FBE563f87BF>

In [6]:
slot0 = pool.slot0()
slot0

slot0_return(sqrtPriceX96=78233251378476677877191459678, tick=-253, observationIndex=0, observationCardinality=1, observationCardinalityNext=1, feeProtocol=0, unlocked=True)

In [7]:
1.0001 ** slot0.tick  # looks good from papr.wtf UI

0.9750185962548082

In [8]:
controller = Contract("0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0")
controller

<PaprController 0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0>

In [9]:
controller.target() / 1e18  # looks good from papr.wtf UI

0.9951899187476195

In [10]:
# check controller has same pool as ref
pool.address == controller.pool()

True

In [11]:
# some params we'll need from papr
dt = chain.blocks.head.timestamp - controller.lastUpdated()
target = controller.target()
ltv_max = controller.maxLTV()
funding_period = controller.fundingPeriod()
cap_tm = int(3e18)  # internal constant

In [12]:
# print out papr params
print(f"PAPR params\ndt: {dt}\ntarget: {target}\nltv_max: {ltv_max}\nfunding_period: {funding_period}\ncap_tm: {cap_tm}")

PAPR params
dt: 9732
target: 995189918747619582
ltv_max: 500000000000000000
funding_period: 7776000
cap_tm: 3000000000000000000


In [13]:
deploy_block = 16592385  # block at which controller deployed

In [14]:
# find the PAPR vault with the highest LTV
#  1. gather all (account, nft) combos since deploy block
#  2. call vaultInfo view for each to get current status
%time query_debts = controller.IncreaseDebt.query('*', start_block=deploy_block)

INFO: Cache database has not been initialized
CPU times: user 6.11 s, sys: 807 ms, total: 6.92 s
Wall time: 51.7 s


In [15]:
# utility functions to unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict) -> tp.Any:
    return event_args[key]

def unfold(df: pd.DataFrame):
    if df.empty:
        return
    for key in df.iloc[-1]['event_arguments'].keys():
        df[key] = df['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [16]:
unfold(query_debts)

In [17]:
query_debts

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,account,collateralAddress,amount
0,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xbc3ed6B537f2980e66f396Fe14210A5...,0x5287d7960ff46f533dfe74840058fa6cb155433d75ef...,16592714,0x00c1594904a72f8fc22869e2e66149d975930145ec97...,514,177,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,11560000000000000
1,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xCC692D6E11268B40A1E3C58e3D86Fc4...,0x4cb9d0e1dbf432e6dd1875fc6d76718c5771b46159a3...,16592815,0x3afd3631f14f5664ccde758f7c788358f5d7606f2ba9...,298,71,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,500000000000000000
2,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xbc3ed6B537f2980e66f396Fe14210A5...,0x8cb24f69414da14858d89d08cc46c03e32ff2d699b61...,16592963,0x25567545b30c47d5d858d3bff3b842c8fc54377b5cea...,431,204,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,11550000000000000
3,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0x82c1B61dA09b5FDce098a212Bb80702...,0xaac96e379eadd3ec5a634dff5832cd19e1f03050e6bd...,16592993,0xad684acb28b8631ea22c904d53141c888a7d97ff0f3b...,509,453,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,446220000000000000
4,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xbc3ed6B537f2980e66f396Fe14210A5...,0xfb28b13ca4400c6028de057265b7466cd4c01d7e8eaf...,16593033,0xe1d5035cb57e00a9b6e2648915420dbc2be9c9f18cb8...,193,107,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,282510000000000000
...,...,...,...,...,...,...,...,...,...,...,...
102,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0x540E9F78D6265AaA9ad320F31AC38fF...,0xa0bd67335574fde3a65c52bc1d4da9881e6548a30ad5...,16726848,0x85d61eb184a8b1a69831e30745e1ff59960cc94fce9a...,146,52,0x540E9F78D6265AaA9ad320F31AC38fF2553f6F5F,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,200000000000000000
103,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xBC1159Fcaa1Ac43E9A12D3F9e3FbE32...,0xedf8c3542274e441b0ea33bb2ee819e9389e3c885a9a...,16727597,0x716703cc7a670c83d1bb73e8ee12529b68f6ecb363eb...,177,80,0xBC1159Fcaa1Ac43E9A12D3F9e3FbE32B9A676701,0x5Af0D9827E0c53E4799BB226655A1de152A425a5,247560000000000000
104,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xEEd7e1c7c9c68611584e8f6230178Ce...,0xe40dd906aecafea2a245620bcd60f338f4d7d79c795b...,16735253,0xb2f0718c09c9ea03fc2a6a3458d630de2d29e70b791d...,169,69,0xEEd7e1c7c9c68611584e8f6230178Ce0A48Eef7F,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,489415619064836550
105,IncreaseDebt,0x3b29c19ff2fcEa0Ff98D0ef5B184354D74eA74b0,{'account': '0xe73B26fB7C84e4DE41703eD1A76EbF2...,0x57288feb829ace5e1ddf42d9cd419c5be5fffe11346f...,16735950,0xf00ad93463bc8328e1bd401563299903253483449ef4...,327,134,0xe73B26fB7C84e4DE41703eD1A76EbF298638b57e,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,24673700344716807


In [18]:
# filter for unique (account, collateralAddress) combos
query_debts_grouped = query_debts.groupby(["account", "collateralAddress"]).size()
query_debts_grouped

account                                     collateralAddress                         
0x028B2b9adCa1BD1263086b0B221713A684968817  0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB    1
0x0EeDd9117f6b4B36be52F3368FD78392827C3E41  0x1A92f7381B9F03921564a437210bB9396471050C    1
0x0e79c644eC160Afe87eA8e87201C0fe35bA63E1f  0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42    1
0x13eEf4EF8FCa471f242ab0F8F49A3dB6017aDA33  0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD    1
0x3347efC95a206F01653b1a5E29771394421e73cD  0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7    2
                                                                                         ..
0xe70F46dB43085C4dA54Fe380ac04cB3955B50AB3  0x79FCDEF22feeD20eDDacbB2587640e45491b757f    1
0xe73B26fB7C84e4DE41703eD1A76EbF298638b57e  0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD    1
0xfa071DCf1FA4e777a9fAbFB6383Fb54e73B9D545  0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6    1
                                            0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cd

In [19]:
vault_params = query_debts_grouped.index.tolist()
vault_params[:3]

[('0x028B2b9adCa1BD1263086b0B221713A684968817',
  '0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB'),
 ('0x0EeDd9117f6b4B36be52F3368FD78392827C3E41',
  '0x1A92f7381B9F03921564a437210bB9396471050C'),
 ('0x0e79c644eC160Afe87eA8e87201C0fe35bA63E1f',
  '0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42')]

In [20]:
# get all vaults
%time vaults = [{"account": params[0], "collateralAddress": params[1]} | controller.vaultInfo(*params).__dict__ for params in vault_params]

CPU times: user 1.34 s, sys: 114 ms, total: 1.45 s
Wall time: 24.3 s


In [21]:
vaults[:3]

[{'account': '0x028B2b9adCa1BD1263086b0B221713A684968817',
  'collateralAddress': '0x42069ABFE407C60cf4ae4112bEDEaD391dBa1cdB',
  'count': 2,
  'auctionCount': 0,
  'latestAuctionStartTime': 0,
  'debt': 1258710000000000000},
 {'account': '0x0EeDd9117f6b4B36be52F3368FD78392827C3E41',
  'collateralAddress': '0x1A92f7381B9F03921564a437210bB9396471050C',
  'count': 0,
  'auctionCount': 0,
  'latestAuctionStartTime': 0,
  'debt': 0},
 {'account': '0x0e79c644eC160Afe87eA8e87201C0fe35bA63E1f',
  'collateralAddress': '0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42',
  'count': 2,
  'auctionCount': 0,
  'latestAuctionStartTime': 0,
  'debt': 469689466053851300}]

In [22]:
# get the collateral values (in PAPR) for each vault
%time vaults = [ vault | {"collateral": int(vault["count"] * controller.cachedPriceForAsset(vault["collateralAddress"]).price * 1e18 / target)} for vault in vaults]

CPU times: user 1.34 s, sys: 109 ms, total: 1.45 s
Wall time: 22.2 s


In [23]:
# go to pandas df to make life easier
df_vaults = pd.DataFrame(data=vaults)

In [24]:
df_vaults = df_vaults[df_vaults["collateral"] > 0]  # only look at vaults with collateral

In [25]:
df_vaults["ltv"] = df_vaults["debt"] / df_vaults["collateral"]  # calc the ltvs for each

In [26]:
# sort by highest ltv
df_vaults_sorted = df_vaults.sort_values(by='ltv', ascending=False)
df_vaults_sorted

,account,collateralAddress,count,auctionCount,latestAuctionStartTime,debt,collateral,ltv
47,0xEEd7e1c7c9c68611584e8f6230178Ce0A48Eef7F,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,1,0,0,489415619064836550,939376122723188224,0.521001
49,0xad387592c2De9645c3F42B5732B93833e312bD0C,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,535900000000000000,1320347981070364928,0.405878
45,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,19,0,0,750000000000000000,1875229673996522496,0.399951
57,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,1,0,0,220330000000000000,560646756452419648,0.392993
55,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,504460000000000000,1320347981070364928,0.382066
41,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,500000000000000000,1320347981070364928,0.378688
52,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6,1,0,0,472020000000000000,1276896274832807680,0.369662
26,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,484140000000000000,1320347981070364928,0.366676
53,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,2,0,0,687320000000000000,1878752245446376448,0.365839
51,0xbF8060106D2e83C106915A575BaeA3dc90c892a6,0xBd3531dA5CF5857e7CfAA92426877b022e612cf8,2,0,0,3581400000000000000,10207297932422205440,0.350867


In [27]:
df_vaults_sorted['HF'] = (ltv_max / 1e18) / df_vaults_sorted['ltv']  # HF = LTV_max / LTV

In [28]:
df_vaults_sorted  # HF < 1 means liquidatable

,account,collateralAddress,count,auctionCount,latestAuctionStartTime,debt,collateral,ltv,HF
47,0xEEd7e1c7c9c68611584e8f6230178Ce0A48Eef7F,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,1,0,0,489415619064836550,939376122723188224,0.521001,0.959692
49,0xad387592c2De9645c3F42B5732B93833e312bD0C,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,535900000000000000,1320347981070364928,0.405878,1.231898
45,0xE5501BC2B0Df6D0D7daAFC18D2ef127D9e612963,0xCa7cA7BcC765F77339bE2d648BA53ce9c8a262bD,19,0,0,750000000000000000,1875229673996522496,0.399951,1.250153
57,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0xFF9C1b15B16263C61d017ee9F65C50e4AE0113D7,1,0,0,220330000000000000,560646756452419648,0.392993,1.272289
55,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,504460000000000000,1320347981070364928,0.382066,1.308675
41,0xCC692D6E11268B40A1E3C58e3D86Fc4CAAb9b77a,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,500000000000000000,1320347981070364928,0.378688,1.320348
52,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x1CB1A5e65610AEFF2551A50f76a87a7d3fB649C6,1,0,0,472020000000000000,1276896274832807680,0.369662,1.352587
26,0x82c1B61dA09b5FDce098a212Bb8070210AB91049,0x79FCDEF22feeD20eDDacbB2587640e45491b757f,1,0,0,484140000000000000,1320347981070364928,0.366676,1.363601
53,0xbc3ed6B537f2980e66f396Fe14210A56ba3f72C4,0x521f9C7505005CFA19A8E5786a9c3c9c9F5e6f42,2,0,0,687320000000000000,1878752245446376448,0.365839,1.366723
51,0xbF8060106D2e83C106915A575BaeA3dc90c892a6,0xBd3531dA5CF5857e7CfAA92426877b022e612cf8,2,0,0,3581400000000000000,10207297932422205440,0.350867,1.425043
